In [2]:
import pandas as pd

In [6]:
from functools import reduce
from pathlib import Path

In [7]:
?reduce

Docstring:
reduce(function, iterable[, initial]) -> value

Apply a function of two arguments cumulatively to the items of a sequence
or iterable, from left to right, so as to reduce the iterable to a single
value.  For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
of the iterable in the calculation, and serves as a default when the
iterable is empty.
Type:      builtin_function_or_method

In [34]:
file_path = Path('../../datasets/Ch2/yellow_tripdata_2023.csv')

In [105]:
from timeit import timeit

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 14.1 µs


In [108]:
def test1():
    df_pd = pd.read_csv(file_path, memory_map=False, low_memory=False, chunksize=10_000)
    
    results = []
    for chunk in df_pd:
        chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']] = chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].apply(pd.to_datetime)
        chunk['time_delta'] = chunk['tpep_dropoff_datetime'] - chunk['tpep_pickup_datetime']
        chunk['year_month'] = chunk['tpep_dropoff_datetime'].dt.year.astype('str') + chunk['tpep_dropoff_datetime'].dt.month.astype('str')
        chunk['day'] = chunk['tpep_dropoff_datetime'].dt.date
        chunk['year'] = chunk['tpep_dropoff_datetime'].dt.year
        columns = ['PULocationID', 'time_delta', 'year_month', 'trip_distance', 'fare_amount', 'day', 'year', 'passenger_count']
        
        df = chunk[columns]
    
    
    
        momthly_performance = (
        df.groupby(
            ['PULocationID','day','year_month']
            )[['trip_distance', 
               'fare_amount', 
               'passenger_count']]
                     .sum()
                     .groupby(
                    ['year_month']
                        )
                         .agg(['sum', 'count'])
        )
    
        results.append(momthly_performance)

In [109]:
def test2():
    df_pd = pd.read_csv(file_path, memory_map=True, low_memory=False, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

    df_pd[['tpep_pickup_datetime', 'tpep_dropoff_datetime']] = df_pd[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].apply(pd.to_datetime)
    
    df_pd['time_delta'] = df_pd['tpep_dropoff_datetime'] - df_pd['tpep_pickup_datetime']
    df_pd['year_month'] = df_pd['tpep_dropoff_datetime'].dt.year.astype('str') + df_pd['tpep_dropoff_datetime'].dt.month.astype('str')
    df_pd['day'] = df_pd['tpep_dropoff_datetime'].dt.date
    df_pd['year'] = df_pd['tpep_dropoff_datetime'].dt.year
    
    columns = ['PULocationID', 'time_delta', 'year_month', 'trip_distance', 'fare_amount', 'day', 'year', 'passenger_count']
    
    
    
    df = df_pd[columns]
    
    
    momthly_performance = (
    df.groupby(
        ['PULocationID','day','year_month']
        )[['trip_distance', 
           'fare_amount', 
           'passenger_count']]
                 .sum()
                 .groupby(
                ['year_month']
                    )
                     .mean()
    )

In [115]:
%time test1()

/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_6338/3601371737.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']] = chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].apply(pd.to_datetime)
/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_6338/3601371737.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']] = chunk[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].apply(pd.to_datetime)
/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_6338/3601371737.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`.

CPU times: user 32.8 s, sys: 2.83 s, total: 35.7 s
Wall time: 36 s


In [116]:
%time test2()

CPU times: user 34.3 s, sys: 2.8 s, total: 37.1 s
Wall time: 37.6 s


In [93]:
len(results)

1619

In [94]:
results[0].head()

trip_distance       fare_amount       passenger_count      
                     sum count         sum count             sum count
year_month                                                            
20091              18.69     1       70.00     1             1.0     1
20234               3.54     2       20.00     2             2.0     2
20235           61015.49   186   272919.74   186         12678.0   186

In [95]:
results[1].head()

trip_distance       fare_amount       passenger_count      
                     sum count         sum count             sum count
year_month                                                            
20235           33372.72   156   192872.96   156         12445.0   156

In [96]:
results[2].head()

trip_distance       fare_amount       passenger_count      
                     sum count         sum count             sum count
year_month                                                            
20235           36037.16   149    205291.9   149         12806.0   149

In [87]:
8.10+18.47

26.57

In [41]:
?reduce

Docstring:
reduce(function, iterable[, initial]) -> value

Apply a function of two arguments cumulatively to the items of a sequence
or iterable, from left to right, so as to reduce the iterable to a single
value.  For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
of the iterable in the calculation, and serves as a default when the
iterable is empty.
Type:      builtin_function_or_method

In [101]:
final = reduce(lambda x1, x2: x1.add(x2, fill_value=0), results)

In [86]:
final

trip_distance       fare_amount       passenger_count  \
                                  sum count         sum count             sum   
PULocationID year_month                                                         
1            20235              26.57   2.0      237.10   2.0             4.0   
3            20235               0.00   1.0       21.50   1.0             1.0   
4            20235              35.43   3.0      223.00   3.0            16.0   
5            20235              18.00   1.0       67.20   1.0             1.0   
6            20235               0.28   1.0        3.70   1.0             1.0   
...                               ...   ...         ...   ...             ...   
261          20235             619.92   3.0     2777.90   3.0           163.0   
262          20235            1617.03   3.0     9291.61   3.0           740.0   
263          20235            1607.53   4.0     9252.80   4.0           788.0   
264          20235            1287.97   3.0     6796.00   3.0           347.0   
265          20235              58.06   3.0     2879.45   3.0            44.0   

                               
                        count  
PULocationID year_month        
1            20235        2.0  
3            20235        1.0  
4            20235        3.0  
5            20235        1.0  
6            20235        1.0  
...                       ...  
261          20235        3.0  
262          20235        3.0  
263          20235        4.0  
264          20235        3.0  
265          20235        3.0  

[210 rows x 6 columns]

In [103]:
momthly_performance

,trip_distance,fare_amount,passenger_count
year_month,,,
20011,16.190000,68.275000,2.000000
200212,3.656667,18.400000,1.000000
20031,31.340000,146.066667,2.666667
200812,18.003333,74.766667,2.000000
20091,7.271429,31.571429,1.285714
201411,6.430000,33.100000,2.000000
202210,1.197778,73.222222,1.444444
202212,1.810000,11.691667,1.000000
20231,1740.026905,8307.067427,601.936264


In [104]:
final

trip_distance           fare_amount          passenger_count  \
                     sum    count          sum    count             sum   
year_month                                                                
20011              64.76      6.0       273.10      6.0             8.0   
200212             10.97      3.0        55.20      3.0             3.0   
20031              94.02      5.0       438.20      5.0             8.0   
200812             54.01      6.0       224.30      6.0             6.0   
20091              50.90      9.0       221.00      9.0             9.0   
201411              6.43      1.0        33.10      1.0             2.0   
202210             10.78     11.0       659.00     11.0            13.0   
202212             21.72     12.0       140.30     12.0            12.0   
20231        11793902.36  46033.0  56305303.02  46033.0       4079924.0   
20232        11270918.54  43671.0  53091167.36  43671.0       3838211.0   
20233        13281682.81  51688.0  64330050.02  51688.0       4479672.0   
20234        13471624.51  50113.0  63678833.15  50113.0       4423185.0   
20235        15271200.17  52710.0  69842796.34  52710.0       4635809.0   
20236            7366.62    220.0     33231.12    220.0          1329.0   
20237              16.07      8.0       106.80      8.0             9.0   
20238               6.00      2.0        33.30      2.0             2.0   
20239               0.45      1.0         5.80      1.0             1.0   

                     
              count  
year_month           
20011           6.0  
200212          3.0  
20031           5.0  
200812          6.0  
20091           9.0  
201411          1.0  
202210         11.0  
202212         12.0  
20231       46033.0  
20232       43671.0  
20233       51688.0  
20234       50113.0  
20235       52710.0  
20236         220.0  
20237           8.0  
20238           2.0  
20239           1.0